## Using Validation/Cross-Validation For Model Selection

This notebook demonstrates some typical workflows for using validation data to select models. It also demonstrates the use of some utility methods like generating **polynomial features**, converting **categorical features to "dummy variable"** binary columns, and **scaling features** when applying regularization.

In [1]:
#Data loading: cars data set (using car characteristics to predict the price)
import pandas as pd
import numpy as np

df=pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data', header=None)

columns= ['symboling','normalized-losses','make','fuel-type',
          'aspiration','num-of-doors','body-style','drive-wheels',
          'engine-location','wheel-base','length','width','height',
          'curb-weight','engine-type','num-of-cylinders','engine-size',
          'fuel-system','bore','stroke','compression-ratio','horsepower',
          'peak-rpm','city-mpg','highway-mpg','price']
df.columns=columns

In [2]:
df.head(3)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500


We're going to simplify things a bit by focusing on the numeric columns and a single categorical column, make.

In [3]:
#simple cleaning
df = df.replace('?', np.NaN).dropna().reset_index()
df['price'] = df['price'].astype(float)
cars = df.select_dtypes(exclude=['object'])

cars['make'] = df['make']
cars.head(3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,index,symboling,wheel-base,length,width,height,curb-weight,engine-size,compression-ratio,city-mpg,highway-mpg,price,make
0,3,2,99.8,176.6,66.2,54.3,2337,109,10.0,24,30,13950.0,audi
1,4,2,99.4,176.6,66.4,54.3,2824,136,8.0,18,22,17450.0,audi
2,6,1,105.8,192.7,71.4,55.7,2844,136,8.5,19,25,17710.0,audi


A machine learning model has no idea how to interpret something like 'audi', it only understands numbers! A very common trick for addressing this is the use of **"dummy variables"**. 

Dummy variables are binary feature columns corresponding to each category. The value is 1 if the observation is in that category, and 0 if not. We can then just use these columns as features in our regression model.

Pandas makes it very easy for us to make this conversion. Notice that the new dummies df has 18 columns, 1 for each category.

In [4]:
cars['make'].nunique()

18

In [5]:
pd.get_dummies(cars['make']).head(5)

,audi,bmw,chevrolet,dodge,honda,jaguar,mazda,mercedes-benz,mitsubishi,nissan,peugot,plymouth,porsche,saab,subaru,toyota,volkswagen,volvo
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
cars = pd.get_dummies(cars) #can just apply it to the whole df
cars.head(3)

,index,symboling,wheel-base,length,width,height,curb-weight,engine-size,compression-ratio,city-mpg,...,make_mitsubishi,make_nissan,make_peugot,make_plymouth,make_porsche,make_saab,make_subaru,make_toyota,make_volkswagen,make_volvo
0,3,2,99.8,176.6,66.2,54.3,2337,109,10.0,24,...,0,0,0,0,0,0,0,0,0,0
1,4,2,99.4,176.6,66.4,54.3,2824,136,8.0,18,...,0,0,0,0,0,0,0,0,0,0
2,6,1,105.8,192.7,71.4,55.7,2844,136,8.5,19,...,0,0,0,0,0,0,0,0,0,0


Now we're ready to start modeling! We're going to try out the validation process to choose between 3 models: simple linear regression, linear regression with ridge regularization, and linear regression with 2nd degree polynomial features.

## Method 1: Train / Validation / Test

Here we will break the data into 3 portions: 60% for training, 20% for validation (used to select the model), 20% for final testing evaluation.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

X, y = cars.drop('price',axis=1), cars['price']

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10) #hold out 20% of the data for final testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3) #hold out 20% for validation

Now we need some setup: **when using regularization, we must standardize** the data so that all features are on the same scale (we subtract the mean of each column and divide by the standard deviation, giving us features with mean 0 and std 1). Since this scaling is part of our model, we need to scale using the training set feature distributions and apply the same scaling to validation and test without refitting the scaler. 

Also, we need to get **polynomial features** for the poly model

In [8]:
#set up the 3 models we're choosing from:

lm = LinearRegression()

#Feature scaling for train, val, and test so that we can run our ridge model on each
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

lm_reg = Ridge(alpha=1)

#Feature transforms for train, val, and test so that we can run our poly model on each
poly = PolynomialFeatures(degree=2) 

X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.transform(X_val)
X_test_poly = poly.transform(X_test)

lm_poly = LinearRegression()

Now we can train, validate, and test.

In [9]:
#validate

lm.fit(X_train, y_train)
print('Linear Regression val R^2: %.3f', lm.score(X_val, y_val))

lm_reg.fit(X_train_scaled, y_train)
print('Ridge Regression val R^2: %.3f', lm_reg.score(X_val_scaled, y_val))

lm_poly.fit(X_train_poly, y_train)
print('Degree 2 polynomial regression val R^2: %.3f', lm_poly.score(X_val_poly, y_val))

Linear Regression val R^2: %.3f 0.8351448283609554
Ridge Regression val R^2: %.3f 0.7367998108560483
Degree 2 polynomial regression val R^2: %.3f -5.136072921094766


/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


Check out that negative R^2, some severe overfitting! 

So having run this validation step, we see that the evidence points to simple linear regression being the best model. So our validation process lets us **select** that choice of model, and as our final step we retrain it on the entire chunk of train/val data and see how it does on test data:  

In [10]:
lm.fit(X,y)
print('Linear Regression test R^2: %.3f', lm.score(X_test, y_test))

Linear Regression test R^2: %.3f 0.9058588914828554


Not terrible!

This is a pretty solid selection method, but we can make it even more rigorous using **cross-validation**.

## Method 2: Cross-Validation / Test

Here we will break the data into 2 portions: 80% for a cross-validated training process, and 20% for final testing evaluation. 

Remember that the idea of CV is to make efficient use of the data available to us (using 80% instead of 60% above), while also performing multiple validation checks. For k-fold CV, we come up with k train/validation splits of the whole chunk of data, in such a way that **each observation is in the validation set exactly 1 time**. Here's a helpful diagram:

![](images/07_cross_validation_diagram.png)

For simplicity we'll focus on linear regression and ridge regression (we also can feel pretty comfortable throwing out the full degree 2 polynomial regression based on the poor results above!) As we loop through our CV folds, we will train and validate both models and collect the results to compare at the end. Note that we scale the training features within the CV loop.

In [11]:
from sklearn.model_selection import KFold

X, y = cars.drop('price',axis=1), cars['price']

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10) #hold out 20% of the data for final testing

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

In [12]:
#run the CV

kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print('Simple mean cv r^2: %.3f +- %.3f' %(np.mean(cv_lm_r2s),np.std(cv_lm_r2s)))
print('Ridge mean cv r^2: %.3f +- %.3f' %(np.mean(cv_lm_reg_r2s),np.std(cv_lm_reg_r2s)))

Simple regression scores:  [0.947403909584367, 0.8724453019892945, 0.41023920794931845, 0.9327397319071882, 0.8358438145266958]
Ridge scores:  [0.93307671766577, 0.8577818780715674, 0.8316030184988574, 0.9084829083232397, 0.845830281440404] 

Simple mean cv r^2: 0.800 +- 0.199
Ridge mean cv r^2: 0.875 +- 0.039


The plot thickens! Our simple validation method above pointed to ridge being better than simple linear regression, but CV shows the opposite. The ridge model appears to be both better on average and lower variance.

**Since CV is more reliable than a single validation set, we select the ridge regression model**. This shows the dangers of relying on simple validation methods, especially when our sample sizes are small.

Finally, see that we do better on the same test set!

In [13]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

lm_reg = Ridge(alpha=1)
lm_reg.fit(X_scaled,y)
print('Ridge Regression test R^2: %.3f', lm_reg.score(X_test_scaled, y_test))

Ridge Regression test R^2: %.3f 0.9184181719692978


## CV, in a less manual way

The CV loop we created above required a chunk of code, but we usually expect sklearn to let us make everything much simpler. It turns out we can in this case too, at the expense of fine-grained control over exactly what happens within the CV loop. When we want to scale each training set within the CV loop or apply other transformations, the fine-grained control is nice, but often we can keep things simple and use the below.

In [14]:
from sklearn.model_selection import cross_val_score
lm = LinearRegression()

cross_val_score(lm, X, y, # estimator, features, target
                cv=5, # number of folds 
                scoring='r2') # scoring metric

array([ 0.69607972,  0.8241609 , -1.51543653,  0.86460568, -0.89091017])

We could also recreate the exact partitioning we used for the manual version by passing a KFold object to `cross_val_score` -- note the results are identical to the manual output above.

In [15]:
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cross_val_score(lm, X, y, cv=kf, scoring='r2')

array([0.94740391, 0.8724453 , 0.41023921, 0.93273973, 0.83584381])

And we can quickly add more evidence that the regularized model will generalize better - choosing a new KFold partioning, we can compare the model scores again and see that the ridge model does better, even without scaling.

In [16]:
kf = KFold(n_splits=5, shuffle=True, random_state = 1000)

print(np.mean(cross_val_score(lm, X, y, cv=kf, scoring='r2')))
print(np.mean(cross_val_score(lm_reg, X, y, cv=kf, scoring='r2')))

0.8211251264617905
0.8842435671302435
